REPLICATION OF SECOND CHAPTER SIMULATIONS

In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import zipfile

In [3]:
base_dir = "Insert directory here"
data_dir = os.path.join(base_dir, 'First_Application_Data')
text_files = os.listdir(data_dir)

In [4]:
with zipfile.ZipFile(os.path.join(base_dir, 'CountryZip.zip'), 'r') as myzip:
    myzip.extractall(data_dir)

In [5]:
# Orthonormalise a matrix via the Gram-Schmidt process
def gram_schmidt_process(columns):
    num_columns = columns.shape[1]
    orthonormal_columns = np.zeros((columns.shape[0], num_columns))
    for i in range(num_columns):
        v = columns.iloc[:, i]
        for j in range(i):
            v -= np.dot(v, orthonormal_columns[:, j]) * orthonormal_columns[:, j]
        norm = np.linalg.norm(v)
        if norm > 0:
            orthonormal_columns[:, i] = v / norm       
    return pd.DataFrame(orthonormal_columns, columns = columns.columns)

# Calculate the two-stage least squares estimator
def two_stage_least_squares(X, y, Z):
    Pi2sls = np.linalg.lstsq(Z, X, rcond = None)[0]
    X2sls = (Z@Pi2sls).reshape(-1, 1)
    v2sls = X - Z @ Pi2sls
    b2sls = np.linalg.lstsq(X2sls, y, rcond = None)[0]
    u2sls = y - X * b2sls
    return b2sls, u2sls, X2sls, v2sls

# Calculate the minimum eigenvalue for the limited information maximum likelihood estimator
def fun_min_eig(W, Z, n):
    sigw = (W.T @ W) / n
    sigw12 = scipy.linalg.sqrtm(sigw)
    wpzw = (W.T @ Z) @ (np.linalg.lstsq(Z, W, rcond = None)[0])
    eig_mat = np.linalg.inv(sigw12) @ wpzw @ np.linalg.inv(sigw12) / n
    return np.linalg.eigvals(eig_mat).min()

# Calculate the limited information maximum likelihood estimator
def limited_information_maximum_likelihood(X, Z, W, X2sls, n):
    min_eig = fun_min_eig(W, Z, n)
    bliml = ((X2sls.T @ y - min_eig * (X.T @ y))) / ((X2sls.T @ X - min_eig * (X.T @ X)))
    uliml = (y - X * bliml).reshape(-1, 1)
    Muliml = np.identity(n) - (uliml @ (np.linalg.inv(uliml.T @ uliml)) @ uliml.T)
    Piliml = (np.linalg.inv(Z.T @ Muliml @ Z)) @ (Z.T @ Muliml @ X)
    Xliml = (Z @ Piliml).reshape(-1, 1)
    return bliml, uliml, Xliml

# Calculate the heteroskedasticity-robust score statistic
def fun_score(Z, xhat, uhat):
    Z2 = Z[:,1:]
    mxz = Z2 - (xhat @ (np.linalg.lstsq(xhat, Z2, rcond = None)[0]))
    variance = mxz.T @ (np.diagflat(uhat * uhat)) @ mxz
    return uhat.T @ mxz @ (np.linalg.inv(variance)) @ mxz.T @ uhat

# Calculate the Newey-West heteroskedasticity-autocorrelation-robust variance estimator
def newey_west(e, X, L):
    N, k = X.shape[0], X.shape[1] 
    Q = np.zeros((k, k))
    for l in range(L + 1):
        w_l = 1 - l / (L + 1)
        for t in range(l + 1, N):
            if l == 0:
                Q += e[t] ** 2 * np.outer(X[t, :], X[t, :])
            else:
                Q += w_l * e[t] * e[t - l] * (np.outer(X[t, :], X[t - l, :]) + np.outer(X[t - l, :], X[t, :]))
    return Q

# Calculate the heteroskedasticity-autocorrelation-robust score statistic   
def fun_score_newey_west(Z, xhat, uhat, n, lags):
    Z2 = Z[:,1:]
    mxz = Z2 - xhat @ (np.linalg.lstsq(xhat, Z2, rcond = None)[0])
    variance = newey_west(uhat, mxz, lags)
    return uhat.T @ mxz @ (np.linalg.inv(variance)) @ mxz.T @ uhat

# Calculate the effective F statistic and effective degrees of freedom
def effective_F(X, Z, vhat, lags):
    Fweight = newey_west(vhat, Z, lags)
    Fweight_tr = np.trace(Fweight)
    eff = (X.T @ Z @ Z.T @ X) / Fweight_tr
    effDOF = ((Fweight_tr ** 2) * 21) / ((np.trace(Fweight.T @ Fweight)) + 20 * Fweight_tr * np.max(np.linalg.eigvals(Fweight)))
    return eff, effDOF

# Simulate the critical value for the effective F statistic
def critical_value(Keff, tau):
    np.random.seed(234435)
    vals = np.random.noncentral_chisquare(Keff, tau * Keff, 10000000) / Keff  
    cv = np.percentile(vals, 95)
    return cv

In [10]:
text_file_count = len(text_files)

table_header = ['Country', 'F', 'cv', '2SLS', 'LIML', 'J', 'KP']

table1 = pd.DataFrame(columns = table_header)
table2 = pd.DataFrame(columns = table_header)

for table in range(1,3):

    for country_number in range(0, text_file_count):

        original_df = pd.read_csv(os.path.join(data_dir, os.listdir(data_dir)[country_number]))

        column_name = original_df.columns[0]

        new_column_names = original_df.columns[0].split("\t")
        joined_data = original_df.values

        new_data = []

        for row in range(original_df.shape[0]):
            joined_data = original_df.iloc[row, 0]
            data_list = joined_data.split("\t")
            
            row_data = {column_name: value for column_name, value in zip(new_column_names, data_list)}

            new_data.append(row_data)

        df = pd.DataFrame(new_data)
        df[new_column_names[0]] = df[new_column_names[0]].apply(pd.to_datetime, errors = 'coerce')
        df[new_column_names[1:]] = df[new_column_names[1:]].apply(pd.to_numeric, errors = 'coerce')

        df = df.iloc[2:]
        df = df.reset_index(drop=True)

        if country_number < 10:
            lags = 4
        else:
            lags = 6

        obs = len(df)
        onevec = np.ones((obs, 1))
        M = np.identity(obs) - (onevec @ onevec.T) /obs

        model_vars = ['dc', 'rr', 'rrf', 'z1', 'z2', 'z3', 'z4']
        new_model_vars = [var.upper() for var in model_vars]

        df[new_model_vars] = df[model_vars].apply(lambda x: M @ x)

        df[new_model_vars[-4:]] = gram_schmidt_process(df[new_model_vars[-4:]])
        df[new_model_vars[-4:]] = df[new_model_vars[-4:]] * np.sqrt(obs)

        country_name = text_files[country_number][:-5]
        if len(country_name) == 3:
            pass
        else:
            country_name = ''.join([country_name, ' '])

        if table == 1:
            y = df['DC'].values
            X = df['RRF'].values
            Z = df[new_model_vars[-4:]].values
            W = df[['DC', 'RRF']].values

            tsls_mat = two_stage_least_squares(X, y, Z)
            liml_mat = limited_information_maximum_likelihood(X, Z, W, tsls_mat[2], obs)
            hansen = np.round(fun_score_newey_west(Z, tsls_mat[2], tsls_mat[1], obs, lags),2)
            kp = np.round(fun_score_newey_west(Z, liml_mat[2], liml_mat[1], obs, lags),2)    

            eff, effDOF =  effective_F(X, Z, tsls_mat[3], lags)
            cv = critical_value(effDOF, 10)

            new_row_table = pd.DataFrame({'Country': [country_name], 'F': [eff], 'cv': [cv], '2SLS': [tsls_mat[0][0]], 'LIML': [liml_mat[0][0]], 'J': [hansen], 'KP': [kp][0][0]})
            table1 = pd.concat([table1, new_row_table], ignore_index = True)

        else:
            y = df['RRF'].values
            X = df['DC'].values
            Z = df[new_model_vars[-4:]].values
            W = df[['RRF', 'DC']].values

            tsls_mat = two_stage_least_squares(X, y, Z)
            liml_mat = limited_information_maximum_likelihood(X, Z, W, tsls_mat[2], obs)
            hansen = np.round(fun_score_newey_west(Z,tsls_mat[2], tsls_mat[1], obs, lags),2)
            kp = np.round(fun_score_newey_west(Z, liml_mat[2], liml_mat[1], obs, lags),2)    

            eff, effDOF =  effective_F(X, Z, tsls_mat[3], lags)
            cv = critical_value(effDOF, 10)

            new_row_table = pd.DataFrame({'Country': [country_name], 'F': [eff], 'cv': [cv], '2SLS': [tsls_mat[0][0]], 'LIML': [liml_mat[0][0]], 'J': [hansen], 'KP': [kp][0][0]})
            table2 = pd.concat([table2, new_row_table], ignore_index = True)

        print(f"{country_name} | F: {eff:.2f}, cv: {cv:.2f} 2SLS: {tsls_mat[0][0]:.2f}, LIML: {liml_mat[0][0]:.2f}, Hansen: {hansen}, KP: {kp[0][0]:.2f}")

AUL | F: 19.17, cv: 18.40 2SLS: 0.05, LIML: 0.03, Hansen: 8.71, KP: 8.82
CAN | F: 13.82, cv: 18.58 2SLS: -0.30, LIML: -0.34, Hansen: 5.02, KP: 5.03
FR  | F: 42.09, cv: 19.33 2SLS: -0.08, LIML: -0.08, Hansen: 0.46, KP: 0.46
GER | F: 14.62, cv: 18.60 2SLS: -0.42, LIML: -0.44, Hansen: 2.68, KP: 2.65
ITA | F: 21.37, cv: 18.92 2SLS: -0.07, LIML: -0.07, Hansen: 1.1, KP: 1.09
JAP | F: 5.48, cv: 21.33 2SLS: -0.04, LIML: -0.05, Hansen: 4.57, KP: 4.58
NTH | F: 13.66, cv: 18.41 2SLS: -0.15, LIML: -0.14, Hansen: 3.67, KP: 3.67
SWD | F: 21.11, cv: 18.70 2SLS: -0.00, LIML: -0.00, Hansen: 2.58, KP: 2.58
SWT | F: 8.12, cv: 18.14 2SLS: -0.49, LIML: -0.50, Hansen: 2.28, KP: 2.29
UK  | F: 8.44, cv: 20.12 2SLS: 0.17, LIML: 0.16, Hansen: 5.05, KP: 5.08
USA | F: 8.66, cv: 18.67 2SLS: 0.06, LIML: 0.03, Hansen: 7.22, KP: 7.59
AUL | F: 2.45, cv: 19.46 2SLS: 0.50, LIML: 30.03, Hansen: 9.49, KP: 8.82
CAN | F: 2.96, cv: 18.10 2SLS: -1.04, LIML: -2.98, Hansen: 6.97, KP: 5.03
FR  | F: 0.22, cv: 19.69 2SLS: -3.12, L

In [11]:
full_results = pd.concat([table1, table2], axis = 1)
full_results = full_results.round(2)
full_results

,Country,F,cv,2SLS,LIML,J,KP,Country,F,cv,2SLS,LIML,J,KP
0,AUL,19.17,18.40,0.05,0.03,8.71,8.82,AUL,2.45,19.46,0.50,30.03,9.49,8.82
1,CAN,13.82,18.58,-0.30,-0.34,5.02,5.03,CAN,2.96,18.10,-1.04,-2.98,6.97,5.03
2,FR,42.09,19.33,-0.08,-0.08,0.46,0.46,FR,0.22,19.69,-3.12,-12.38,2.14,0.46
3,GER,14.62,18.60,-0.42,-0.44,2.68,2.65,GER,1.28,20.16,-1.05,-2.29,2.98,2.65
4,ITA,21.37,18.92,-0.07,-0.07,1.10,1.09,ITA,0.49,18.89,-3.34,-14.81,3.99,1.09
5,JAP,5.48,21.33,-0.04,-0.05,4.57,4.58,JAP,2.00,17.89,-0.18,-21.56,8.42,4.58
6,NTH,13.66,18.41,-0.15,-0.14,3.67,3.67,NTH,1.87,18.94,-0.53,-6.94,10.18,3.67
7,SWD,21.11,18.70,-0.00,-0.00,2.58,2.58,SWD,0.90,17.31,-0.10,-399.86,13.27,2.58
8,SWT,8.12,18.14,-0.49,-0.50,2.28,2.29,SWT,1.62,20.00,-1.56,-2.00,2.95,2.29
9,UK,8.44,20.12,0.17,0.16,5.05,5.08,UK,2.68,17.63,1.06,6.21,8.17,5.08
